In [13]:
!pip install flask --no-deps

  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
!pwd

/workspace/CLIP-fine-tune


In [36]:
from PIL import Image
import os

def create_thumbnails(input_dir, output_dir, thumb_size=(64, 64)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".png"):  # Adjust this if your files have a different extension
            image_path = os.path.join(input_dir, filename)
            img = Image.open(image_path)
            img.thumbnail(thumb_size, Image.ANTIALIAS)  # Resize the image

            # Create a new filename for the thumbnail
            base, ext = os.path.splitext(filename)
            new_filename = f"{base}{ext}"
            output_path = os.path.join(output_dir, new_filename)

            # Save the thumbnail
            img.save(output_path)

            print(f"Thumbnail created for {filename} at {output_path}")

# Specify the input and output directories
input_dir = 'tiles'
output_dir = 'tiles/thumbs'

create_thumbnails(input_dir, output_dir)


/tmp/ipykernel_866/4017276661.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img.thumbnail(thumb_size, Image.ANTIALIAS)  # Resize the image


Thumbnail created for 8192_5632.png at tiles/thumbs/8192_5632.png
Thumbnail created for 8192_6656.png at tiles/thumbs/8192_6656.png
Thumbnail created for 2560_5632.png at tiles/thumbs/2560_5632.png
Thumbnail created for 3072_4096.png at tiles/thumbs/3072_4096.png
Thumbnail created for 5120_5632.png at tiles/thumbs/5120_5632.png
Thumbnail created for 3584_6656.png at tiles/thumbs/3584_6656.png
Thumbnail created for 7168_6656.png at tiles/thumbs/7168_6656.png
Thumbnail created for 1024_5632.png at tiles/thumbs/1024_5632.png
Thumbnail created for 4608_512.png at tiles/thumbs/4608_512.png
Thumbnail created for 4608_1024.png at tiles/thumbs/4608_1024.png
Thumbnail created for 4096_1024.png at tiles/thumbs/4096_1024.png
Thumbnail created for 3584_4096.png at tiles/thumbs/3584_4096.png
Thumbnail created for 5632_4608.png at tiles/thumbs/5632_4608.png
Thumbnail created for 3584_3584.png at tiles/thumbs/3584_3584.png
Thumbnail created for 5120_512.png at tiles/thumbs/5120_512.png
Thumbnail crea

In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_bootstrap import Bootstrap
app = Flask(__name__)
Bootstrap(app)

import clip
import torch
from PIL import Image
clipmodel = 'ViT-L/14'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clipmodel, device=device)

# Load the fine-tuned model
model_path = 'models/clip_ft_30.pt'  # Update this path
model = torch.load(model_path)
model.eval()
model_path

import os

def gather_prompts(directory):
    prompts = set()  # Using a set to avoid duplicate prompts

    # Walk through all files in the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):  # Ensure it's a text file
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    # Read each line from the file, strip whitespace, and add to the set
                    for line in f:
                        stripped_line = line.strip()
                        #print(stripped_line)
                        if stripped_line:  # Only add non-empty lines
                            prompts.add(stripped_line)

    return list(prompts)  # Convert set to list to allow indexing

# Specify the path to your directory containing text files
directory_path = 'tiles/'
all_prompts = gather_prompts(directory_path)

print(f"Found {len(all_prompts)} unique prompts.")

# Assuming 'model', 'preprocess', and 'image_preprocessed' are already defined
text_tokens = clip.tokenize(all_prompts).to(device)

# Proceed with model encoding and similarity calculation
with torch.no_grad():
    #image_features = model.encode_image(image_preprocessed)
    text_features = model.encode_text(text_tokens)


@app.route('/')
def display_grid():
    max_x = 8192
    max_y = 7168
    step = 512
    num_tiles_x = max_x // step + 1
    num_tiles_y = max_y // step + 1
    
    tiles = []
    for x in range(num_tiles_x):
        row = []
        for y in range(num_tiles_y):
            tile_path = f'thumbs/{x * step}_{y * step}.png'
            row.append(tile_path)
        tiles.append(row)
    
    return render_template('index.html', tiles=tiles)


@app.route('/update_position', methods=['POST'])
def update_position():
    data = request.get_json()
    x = data.get('x')
    y = data.get('y')
    print(f"Overlay moved to position: ({x*16}, {y*16})")
    image_path = f'tiles/{x*16}_{y*16}.png'  # Adjust this path
    image = Image.open(image_path).convert("RGB")
    
    # Preprocess the image
    image_preprocessed = preprocess(image).unsqueeze(0).to(device)
    
    # Proceed with model encoding and similarity calculation
    with torch.no_grad():
        # Calculate the cosine similarity
        logits_per_image, logits_per_text = model(image_preprocessed, text_tokens)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    
    # Display the top scoring prompts
    top_prompts = sorted(zip(all_prompts, probs[0]), key=lambda x: x[1], reverse=True)[0]
    text, confidence = (top_prompts)
    coordinates = text.split('),')[0]+')'
    description = text.split('),')[1]
    # Here you can do more with the position, like storing it or processing it further.
    return jsonify({'x': x * 16, 'y': y * 16, 'message': description, 'coords': coordinates})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=3002)


Found 255 unique prompts.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3002
 * Running on http://192.168.96.2:3002
Press CTRL+C to quit


Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)


100.64.0.24 - - [05/May/2024 10:55:36] "POST /update_position HTTP/1.1" 400 -
100.64.0.24 - - [05/May/2024 10:55:36] "POST /update_position HTTP/1.1" 200 -


Overlay moved to position: (512, 0)
Overlay moved to position: (512, 0)


...

Overlay moved to position: (4096, 4096)


100.64.0.24 - - [05/May/2024 18:00:31] "GET /static/tiles/4096_3584.png HTTP/1.1" 200 -
100.64.0.24 - - [05/May/2024 18:00:31] "POST /update_position HTTP/1.1" 200 -
100.64.0.24 - - [05/May/2024 18:00:32] "GET /static/tiles/4096_4096.png HTTP/1.1" 304 -


Overlay moved to position: (4096, 4096)


100.64.0.24 - - [05/May/2024 18:00:42] "POST /update_position HTTP/1.1" 200 -


Overlay moved to position: (4096, 4096)
Overlay moved to position: (4096, 4096)


In [67]:
!pip install flask-bootstrap --no-deps

  Using cached Flask_Bootstrap-3.3.7.1-py3-none-any.whl

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
